In [195]:
import pandas as pd
from openai import OpenAI
client = OpenAI(project = 'proj_gEusaRT8EMi1zd7hpRVJ4VV5')

data = pd.read_csv('~/Documents/SOCY3001/ResearchMapping/rawdata2.csv')
data = data
data

output = pd.DataFrame(columns=['Location', 'Data', 'Goal', 'Coordinates'])
output = []

In [196]:
for i in range(len(data)):
    prompt = f"Given the following research description, provide the following information:\n\
    1) The location where the research was conducted, if specified. If it is completely impossible to say, respond with 'Not specified'\n\
    2) The type of data used in the research (e.g. satellite imagery, remote sensing data). If it is completely impossible to even make a guess, respond with 'Not specified'\n\
    3) The type/goal of the research (e.g. predicting forest fires, classifying tree species).\n\
    4) Your estimation of the coordinates where the research was conducted, separated by ', '. Only provide one set of coordinates. If the location is not specified, or the research is being conducted on global scale and/or is not being conducted in a specific location, respond with 'Not specified'. If only a country/general region is specified, make your best guess at the approximate coordinates or pick a random set of coordinates within that region. The coordinates should be formatted as XX.XXXXXX, XX.XXXXXXX (e.g. 12.345678, 12.345678). Do not use brackets, letters, extra spaces or punctuation, or anything else. You must format the coordinates perfectly - this is absolutely critical, as otherwise everything will break. \n\
    If the research is not related in any way whatsoever to using artificial intelligence/machine learning/deep learning to study forests/trees/woodlands, even slightly, return N/A for all questions. \n\\n\
    Give your response as exactly FOUR values separated by semicolons. Each response MUST have exactly three semicolons - if you don't use exactly 3 semicolons, everything will break, so getting the formatting correct is of critical importance. (e.g. China;Satellite images;Forest fire detection;41.40338, 2.17403).\n\
    Where possible, the data used and the goal of the research should be general and capable of being aggregated with other, similar research (e.g. 'Remote sensing data' or 'Tree species classification').\n\n\
    Research description:\n{data.iloc[i,0]}\n\
    {data.iloc[i,1]}\n\n\
    {data.iloc[i,2]}"
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt}
            ],
        }
        ]
    )
#     col1, col2, col3, col4 = response.choices[0].message.content.split(';')
#     new_row = pd.DataFrame({'Location': [col1], 'Data': [col2], 'Goal': [col3], 'Coordinates': [col4]})
#     output = pd.concat([output, new_row], ignore_index=True)
    output.append(response.choices[0].message.content.split(';')) 

In [191]:
for inner_list in output:
    if len(inner_list) != 4:
        print(inner_list)

In [192]:
output_clean = output
output_df = pd.DataFrame(output_clean, columns=['Location', 'Data', 'Goal', 'Coordinates'])

file_path = '~/Documents/SOCY3001/ResearchMapping/output.csv'
output_df.to_csv(file_path, index=False)

In [193]:
output2 = []

for i in range(len(output_df)):
    prompt = f"For the following study, respond with the category it best fits into from the following options:\n\
    Tree species classification \n\
    Forest cover assessment \n\
    Canopy and biomass estimation \n\
    Habitat degradation and biodiversity monitoring \n\
    Forest fire monitoring and prevention \n\
    Other \n\
    N/A \n\
    Respond with the category ONLY, exactly as written in the prompt. DO NOT INCLUDE NUMBERS, other commentary, or anything else.\n\n\
    {output_df.iloc[i,2]}"
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt}
            ],
        }
        ]
    )
#     col1, col2, col3, col4 = response.choices[0].message.content.split(';')
#     new_row = pd.DataFrame({'Location': [col1], 'Data': [col2], 'Goal': [col3], 'Coordinates': [col4]})
#     output = pd.concat([output, new_row], ignore_index=True)
    output2.append(response.choices[0].message.content) 

In [194]:
names = pd.read_csv('~/Documents/SOCY3001/ResearchMapping/rawdata.csv').iloc[:,8]

output_df['Category'] = output2
output_df['Title'] = names
output_df_to_save = output_df[(output_df != "N/A").all(axis=1)]
output_df_to_save.to_csv(file_path, index=False)
output_df_to_save

,Location,Data,Goal,Coordinates,Category,Title
0,Brussels Capital Region,Remote sensing data,Tree species classification,"50.85034, 4.35171",Tree species classification,Fusion of multi-temporal PlanetScope data and ...
1,Sundarbans,Remote sensing data,Habitat quality analysis,"22.17204, 88.17421",Habitat degradation and biodiversity monitoring,Habitat quality and degradation change analysi...
3,Luoyang,Vegetation indices and canopy height data,Canopy height estimation,"34.60031, 112.45333",Canopy and biomass estimation,Forest Canopy Height Retrieval Model Based on ...
5,Not specified,In-situ microclimate and land cover dataset,Urban land cover influence on microclimate,"37.09024, -95.71289",Other,Urban tree cover provides consistent mitigatio...
7,China,Remote sensing data,Tree species classification,"30.96450, 111.35000",Tree species classification,Remote Sensing Classification and Mapping of F...
...,...,...,...,...,...,...
909,Iran,Remote sensing data,Estimating forest canopy cover,"33.55812, 47.40623",Canopy and biomass estimation,Estimation of Semiarid Forest Canopy Cover Usi...
910,Northern United States,Landsat time series data,Estimating forest carbon stocks,"47.65000, -90.00000",Canopy and biomass estimation,Contemporary forest carbon dynamics in the nor...
911,Germany,South Africa,Remote sensing data,"51.16569, 10.45153",Other,Predicting Forest Cover in Distinct Ecosystems...
912,Finland,Field data and forest resource data,Exploratory statistical analysis and predictiv...,"61.92411, 25.74815",Other,Fusion of open forest data and machine fieldbu...
